Lets build a model!

In [ ]:
!wget https://download.blender.org/release/Blender2.82/blender-2.82a-linux64.tar.xz blender-2.82a-linux64.tar.xz

In [ ]:
!tar xvf blender-2.82a-linux64.tar.xz


In [ ]:
!cd blender-2.82a-linux64/2.82/python/bin && ./python3.7m -m ensurepip && ./python3.7m pip3 install opencv-python && ./python3.7m pip3 install --upgrade Pillow && ./python3.7m pip3 install --upgrade boto3 


In [ ]:
!sudo docker build --tag odin ./

In [ ]:
!sudo docker stop odin-container

In [ ]:

bucket="handtools"
rotation_theta=1
upper_bound=5
rotation_type='X'

import time
timestamp = int(round(time.time() * 1000))


In [ ]:
!echo $timestamp

In [ ]:
!sudo docker run --rm --publish 8080:8080/tcp -e AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID -e AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY --name odin-container odin $rotation_theta $bucket $rotation_type $upper_bound $timestamp >out.log
    
    
    

In [ ]:

%%bash
region=us-east-1
sudo $(aws ecr get-login --region ${region} --no-include-email)

image=odin
account=$(aws sts get-caller-identity --query Account --output text)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"
#aws ecr --region ${region} create-repository --repository-name "${image}"
sudo aws ecr batch-delete-image --repository-name $image --image-ids imageTag=latest
sudo docker tag ${image} ${fullname}
sudo docker push ${fullname}


In [ ]:
!./daemon.sh

In [ ]:
!blender-2.82a-linux64/blender --background -b -noaudio -E CYCLES --python euler_engine.py -- 2 A handtools 1

In [ ]:
#To see logs of the container, run following command.
!docker logs odin-container